# Extract Sentinel-1 Data and Sentinel-2 data for given lat and long

In [ ]:
# Supress Warnings
import warnings
warnings.filterwarnings('ignore')

# Visualization
import ipyleaflet
import matplotlib.pyplot as plt
from IPython.display import Image
import seaborn as sns

# Data Science
import numpy as np
import pandas as pd

# Feature Engineering
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split

# Machine Learning
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import f1_score, accuracy_score,classification_report,confusion_matrix

# Planetary Computer Tools
import pystac
import pystac_client
import odc
from pystac_client import Client
from pystac.extensions.eo import EOExtension as eo
from odc.stac import stac_load
import planetary_computer as pc
#pc.settings.set_subscription_key('******')

# Others
import requests
import rich.table
from itertools import cycle
from tqdm import tqdm
tqdm.pandas()
import os

In [ ]:
# train/test switch
train = False

In [ ]:
os.chdir('/home/jovyan/EY-Crop-identification')
print(os.getcwd())

/home/jovyan/EY-Crop-identification


In [ ]:
if train:
    crop_presence_data = pd.read_csv("Data/Crop_Location_Data_20221201.csv")
else:
    crop_presence_data = pd.read_csv("Data/challenge_1_submission_template_correct_columns_fixed.csv")
crop_presence_data.head()

,Latitude and Longitude,target
0,"(10.18019073690894, 105.32022315786804)",NaN
1,"(10.561107033461816, 105.12772097986661)",NaN
2,"(10.623790611954897, 105.13771401411867)",NaN
3,"(10.583364246115156, 105.23946127195805)",NaN
4,"(10.20744446668854, 105.26844107128906)",NaN


In [ ]:
crop_presence_data.shape

(250, 2)

In [ ]:
def get_sentinel_data(latlong,time_slice,assets):
    '''
    Returns sentinel-1-rt and sentinel-2-l2a for a given latitude and longitude 
    Attributes:
    latlong - A tuple with 2 elements - latitude and longitude
    time_slice - Timeframe for which the VV and VH values have to be extracted
    assets - A list of bands to be extracted
    '''

    latlong=latlong.replace('(','').replace(')','').replace(' ','').split(',')
    bbox_of_interest = (float(latlong[1]) , float(latlong[0]), float(latlong[1]) , float(latlong[0]))
    time_of_interest = time_slice

    catalog = pystac_client.Client.open(
        "https://planetarycomputer.microsoft.com/api/stac/v1"
    )
    search_sen1 = catalog.search(
        collections=["sentinel-1-rtc"], bbox=bbox_of_interest, datetime=time_of_interest
    )
    search_sen2 = catalog.search(collections=["sentinel-2-l2a"], bbox=bbox_of_interest, datetime=time_of_interest)

    items_sen1 = list(search_sen1.get_all_items())

    items_sen2 = list(search_sen2.get_all_items())
    bands_of_interest = assests
    data_sen1 = stac_load(items_sen1, patch_url=pc.sign, bbox=bbox_of_interest)

    resolution = 10  # meters per pixel
    scale = resolution / 111320.0
    data_sen2 = stac_load(items_sen2,
                          bands=["red", "green", "blue", "nir", "SCL"],
                          resolution=scale,
                          bbox=bbox_of_interest,
                          crs="EPSG:4326",
                          patch_url=pc.sign)
    return data_sen1, data_sen2

Extract and save Sentinel-1 Data and Sentinel-2 data for given lat and long

In [ ]:
## Function call to extract VV,VH Values
time_slice = "2021-11-01/2022-12-01"
assests = ['vh','vv']
vhs, vvs = [], []
# set i to the index where the program got disconnected
i = 24
for coordinates in tqdm(crop_presence_data['Latitude and Longitude'][24:]):
    data_sen1, data_sen2 = get_sentinel_data(coordinates,time_slice,assests)
    if train:
        data_sen1.to_netcdf(f"Data/vhvv/train/vhvv_row{i}.nc")
        data_sen2.to_netcdf(f"Data/vhvv/train/vhvv_sen2_row{i}.nc")
    else:
        data_sen1.to_netcdf(f"Data/vhvv/test/vhvv_row{i}.nc")
        data_sen2.to_netcdf(f"Data/vhvv/test/vhvv_sen2_row{i}.nc")
    i += 1

100%|██████████| 226/226 [2:41:50<00:00, 42.97s/it]  
